# MO809 - Lab 02 - Guia para a aula experimental

Agora que você já sabe adquirir dados com um dispositivo e enviar para a nuvem (utilizando a plataforma Konker), vamos implementar o método de aprendizado não supervisionado kmeans no dispositivo, um ESP8266.
Para seguir esse roteiro, você precisará de uma conta na Konker, que pode ser criada em https://demo.konkerlabs.net/ e, caso deseje trabalhar com o kit disponibilizado para vocês, será necessário baixar a Arduino IDE e adicionar o suporte a placa de desenvolvimento **NodeMCU**, além de instalar as bibliotecas **PubSubClient** e **ArduinoJSON**. Um guia de instalação da placa pode ser visto na referência https://www.filipeflop.com/blog/programar-nodemcu-com-ide-arduino/ enquanto a instalação das bibliotecas pode ser feita sem dificuldades buscando pelo nome das mesmas na caixa aberta ao selecionar o menu **Sketch** -> **Incluir Biblioteca** -> **Gerenciar Bibliotecas**.

Vamos iniciar nosso notebook chamando todas as bibliotecas que usaremos. Nessa etapa, nada precisa ser modificado.

In [ ]:
!pip install arrow bokeh paho.mqtt
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import pprint
import numpy as np
import arrow
import requests                                                                                                                                                                                 
import json                                                                                                                                                                                     
from threading import Timer
import matplotlib.pyplot as plt
%matplotlib inline

O próximo passo é definir os endereços que serão usados para consultar e enviar os dados. Nessa etapa ainda não é necessário fazer modificações.

In [ ]:
#Url de publicacao dos dados
pub_url = 'https://data.demo.konkerlabs.net/pub/'
#Url da API
base_api = 'https://api.demo.konkerlabs.net'
#Application padrão
application = 'default'

Agora vamos colocar em duas variáveis o seu usuário e senha da plataforma:

In [ ]:
username = ''
password = ''

O próximo passo é criar o dispositivo dentro de sua conta na Konker. Você pode criar o dispositivo com o nome de sua preferência, lembre-se apenas de modificar nas função abaixo para o nome adequado. Caso queira usar o nome *termometro*, não é necessário alterar nada na próxima célula.

In [ ]:
termometro_name = "termometro"

## Usando o Kit de Hardware 
Esse é o momento de você baixar o código dos dispositivos disponível no GitHub: https://github.com/lfgomez/MO809/lab02/ Você deve baixar o código **kmeans-termistor**. Depois de baixar o código, abra o código do Termometro na Arduino IDE e mude os parâmetros de rede Wifi, canal de publicação (minha sugestão é usar *temperatura*) credenciais do dispositivo.

![term](https://raw.githubusercontent.com/KonkerLabs/arduino_examples/master/Termometro_MQTT/term.jpg "Termômetro")

Com o dispositivo montado, o próximo passo é compilar e gravar o Firmware. Lembre-se de mudar a board na Arduino IDE para **NodeMCU v1.0**.

**Ao iniciar o treinamento, o LED da placa começará a piscar. Após isso, aguarde 10 segundos registrando a temperatura ambiente. Após isso, segure o termistor com seus dedos fazendo a temperatura subir. Mantenha o termômetro aquecido com seus dedos por ao menos 10 segundos. Por último, deixe o dispositivo capturando novamente dados de temperatura ambiente até que o LED da placa pare de piscar.**

**Quando o LED se apagar ou acender continuamente, o treinamento está terminado. A placa então começará a enviar dados para a plataforma, em dois canais: "temperatura" e "cluster". O LED também está configurado para mudar de estado conforme o cluster do dado adquirido: apagado para cluster 0 e aceso para cluster 1.**

## Vamos baixar os dados e ver como eles se comportam
Para iniciar esse trabalho, vamos primeiro conectar na API da Konker. A API usa OAuth2, então primeiro vamos obter as credenciais.

In [ ]:
client = BackendApplicationClient(client_id=username)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url='{}/v1/oauth/token'.format(base_api),
                                       client_id=username,
                                       client_secret=password)

Ótimo! Agora nos podemos começar a usar a API. Caso você queira mais detalhes de sua utilização, pode encontrar documentação em: https://api.demo.konkerlabs.net .

Vamos começar listando os dispositivos registrados no seu usuário.

In [ ]:
devices = oauth.get("https://api.demo.konkerlabs.net/v1/{}/devices/".format(application)).json()['result']
for dev in devices:
    print(dev)

Vamos procurar pelo dispositivo Termometro na sua lista de dispositivos:

In [ ]:
guid_term=""
for dev in devices:
    if dev['name'] == termometro_name:
        guid_term = dev['guid']

print("O GUID do dispositivo Termômetro é: "+ guid_term)

Caso você consiga ver o GUID do dispositivo, significa que está tudo funcionando bem. Caso o GUID não apareça, revise o nome do dispositivo no Notebook e o nome escolhido na plataforma para garantir que eles possuem a mesma grafia.

Agora vamos usar a API para pegar os dados enviados pelo dispositivo termômetro hoje. Vamos baixar dados de todos os canais: "temperatura" e "cluster".

In [ ]:
dt_start = arrow.utcnow().to('America/Sao_Paulo').floor('day')
dt_start = dt_start.shift(days=-1)
stats = oauth.get("https://api.demo.konkerlabs.net/v1/{}/incomingEvents?q=device:{} timestamp:>{}&sort=oldest&limit=10000".format(application,guid_term,dt_start.isoformat())).json()['result']
print(stats)

Caso tudo tenha funcionado como esperado, você deve estar vendo seus dados logo acima. Para facilitar a visualização e análise dos dados, podemos transformar em um formato tabular com o Pandas.

In [ ]:
from pandas import json_normalize
stats_df = json_normalize(stats).set_index('timestamp')
stats_df = stats_df[3:]
stats_df

Ótimo! Agora os dados estão em um formato mais fácil de ler. Vamos agora ver os clusters.

In [ ]:
plt.figure(figsize=(15,4))
stats_df[stats_df['incoming.channel']=='clusters']['payload.value'].plot()
plt.xticks(rotation=45);

Vamos agora plotar os clusters juntamente com a temperatura

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,4))
ax1.set_xlabel('timestamp')
ax1 = stats_df[stats_df['incoming.channel']=='temperatura']['payload.value'].plot()
ax2 = ax1.twinx()
ax2.fill_between(stats_df[stats_df['incoming.channel']=='clusters'].index,stats_df[stats_df['incoming.channel']=='clusters']['payload.value'], 0,color = 'red', alpha=0.3)
ax1.tick_params(labelrotation=45);

## Feito isso, você deve escrever um relatório que contenha: Resumo, metodologia usada e conclusões, respondendo ao longo do texto as perguntas abaixo. 
### Obs.: Essa primeira parte você pode realizar quantas vezes achar necessário para responder as próximas questões.

### Perguntas com relação ao kmeans
#### 1) Como funciona o método de kmeans? Quais os parâmetros do método?
#### 2) Ele é um método supervisionado ou não-supervisionado? O que isso significa?
#### 3) O que você espera como resultado dele de uma forma geral? Ele funciona em dimensões superiores a 3?


### Perguntas com relação ao experimento
#### 1) O que ocorre se você tiver um vetor com todas as temperaturas idênticas?
#### 2) Como você inicializaria os centroids? Consegue apontar como foi feito no código? O que ocorre se forem usadas as duas primeiras entradas do vetor como centroids?
#### 3) O que ocorre se você reduzir o máximo de interações para 2? E para 100000?

### Perguntas com relação a aprendizado no dispositivo
#### 1) Quais as vantagens do aprendizado no dispositivo? (Latência, disponibilidade, privacidade...)
#### 2) Quais são as maiores desvantagens ou dificuldades no aprendizado de máquina em dispositivos?